In [2]:
import pandas as pd 

def get_data_from_google_sheet(sheet_url):
    csv_url = sheet_url.split("/edit")[0] + "/export?format=csv&gid=0"

    try:
        df = pd.read_csv(csv_url)

        expected_columns = ['state_name', 'state_code', 'state_timezone']

        if all(col in df.columns for col in expected_columns):
            df = df[expected_columns]
        else:
            print("Expected columns are not found in the DataFrame")
            return None
        
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

sheet_url = "https://docs.google.com/spreadsheets/d/1I31B5NN7tFYntmATiEE_PiQhZ47DsAXetiDrVsVehfM/edit?pli=1&gid=0#gid=0"
df = get_data_from_google_sheet(sheet_url)

if df is not None:
    print(df.head())

   state_name state_code                                 state_timezone
0     Alabama         AL                              Central Time (CT)
1      Alaska         AK  Alaska Time (AKT), Hawaii-Aleutian Time (HAT)
2     Arizona         AZ                            Mountain Time (MT)*
3    Arkansas         AR                              Central Time (CT)
4  California         CA                              Pacific Time (PT)


In [10]:
df['state_timezone'] = df['state_timezone'].str.replace('*', '', regex=False)

df.head(49)



,state_name,state_code,state_timezone
0,Alabama,AL,Central Time (CT)
1,Alaska,AK,"Alaska Time (AKT), Hawaii-Aleutian Time (HAT)"
2,Arizona,AZ,Mountain Time (MT)
3,Arkansas,AR,Central Time (CT)
4,California,CA,Pacific Time (PT)
5,Colorado,CO,Mountain Time (MT)
6,Connecticut,CT,Eastern Time (ET)
7,Delaware,DE,Eastern Time (ET)
8,Florida,FL,"Eastern Time (ET), Central Time (CT)"
9,Georgia,GA,Eastern Time (ET)


In [12]:
import sqlite3

conn = sqlite3.connect('database.db')
table_name = 'us_state_codes'
identifier_colum = 'state_name'

try:
    existing_df = pd.read_sql_query(f"SELECT *FROM [table_name]", conn)

    merged_df = pd.merge(existing_df, df, on=identifier_colum, how='outer', suffixes=('_old', '_new'))
    
    update_mask = merged_df[identifier_colum].isin(existing_df[identifier_colum]) & merged_df[identifier_colum].isin(df[identifier_colum])

    for col in df.columns:
        if col != identifier_colum and f'{col}_new' in merged_df.colums:
            merged_df.loc[update_mask, col] = merged_df[f'[col]_new']

    final_df = merged_df[df.columns]

except pd.io.sql.DatabaseError:
    final_df = df

final_df.to_sql(table_name, conn, if_exists='replace', index=False)

print(f"Data in table '{table_name}' has been merged/updated")

conn.close()

Data in table 'us_state_codes' has been merged/updated
